<a href="https://colab.research.google.com/github/willystumblr/ict-2023-ems/blob/main/%08ict_2023_model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn, optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [ ]:
import glob
import tqdm
import os
from collections import defaultdict
from datetime import datetime

In [ ]:
PATH = "/content/drive/MyDrive/ict-2023/train"
buildings = defaultdict(pd.DataFrame)
def parser(x):
    return datetime.strptime(x, '%Y %m %d %H')

for path in glob.glob(os.path.join(PATH, "*.csv")):
    building = path.split("/")[-1][:-4]

    data = pd.read_csv(path, index_col=0, parse_dates=[['YEAR', 'MONTH', 'DAY', 'HOUR']], date_parser=parser)
    # dataset = read_csv('raw.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
    t = pd.DataFrame(data.index)
    t_h = pd.DataFrame(data=list(t['YEAR_MONTH_DAY_HOUR'].dt.hour), index=data.index, columns=['Hour'])
    t_m = pd.DataFrame(data=list(t['YEAR_MONTH_DAY_HOUR'].dt.month), index=data.index, columns=['Month'])
    data = pd.concat([data, t_h, t_m], axis=1)
    buildings[building] = data

buildings[building].describe()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),Hour,Month
count,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000,19584.000000
mean,4.376363,161.014042,173.088899,18.004922,16.034860,72.786484,15.899280,10.365125,1007.312199,0.252589,0.643958,5.699653,1601.888583,11.500000,6.557598
std,8.000294,254.056141,274.774182,15.093855,9.784289,22.577161,10.345486,11.702137,8.209513,0.396316,0.932640,3.805323,553.043199,6.922363,3.257671
min,-0.600000,0.000000,0.000000,-19.300000,-13.400000,8.000000,0.600000,-28.800000,975.600000,0.000000,0.000000,0.000000,10.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,6.200000,8.100000,56.000000,6.500000,0.800000,1000.700000,0.000000,0.000000,2.000000,1256.000000,5.750000,4.000000
50%,0.000000,0.000000,0.000000,18.800000,18.000000,76.000000,13.400000,11.300000,1006.700000,0.000000,0.080000,7.000000,1946.000000,11.500000,7.000000
75%,4.600000,245.000000,253.000000,25.800000,24.200000,95.000000,25.000000,21.100000,1013.825000,0.500000,1.080000,9.000000,2000.000000,17.250000,9.000000
max,37.400000,1300.000000,1300.000000,63.300000,34.500000,99.000000,41.900000,29.800000,1027.500000,1.000000,3.880000,10.000000,12000.000000,23.000000,12.000000


# Data preprocessing

We first check the reformatted dataset combined with meteorological data.

In [ ]:
data.head()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),Hour,Month
YEAR_MONTH_DAY_HOUR,,,,,,,,,,,,,,,
2020-06-01 00:00:00,0.0,0.0,0.0,19.0,19.5,89.0,20.1,17.6,1002.9,0.0,0.0,10.0,1652.0,0,6
2020-06-01 01:00:00,0.0,0.0,0.0,18.9,19.3,93.0,20.8,18.1,1003.1,0.0,0.0,10.0,1395.0,1,6
2020-06-01 02:00:00,0.0,0.0,0.0,18.7,19.2,94.0,20.9,18.2,1002.8,0.0,0.0,10.0,1162.0,2,6
2020-06-01 03:00:00,0.0,0.0,0.0,18.7,19.1,96.0,21.2,18.4,1002.7,0.0,0.0,10.0,747.0,3,6
2020-06-01 04:00:00,0.0,0.0,0.0,17.5,18.6,98.0,20.9,18.2,1002.6,0.0,0.0,10.0,509.0,4,6


In [ ]:
data.shape

(19584, 15)

We apply different encoding strategies to features according to their types; MinMaxScaler() for numerical; LabelEncoder() for categorical; and cyclic encoding for cyclic features.

In [ ]:
# Separate numerical and categorical features
numerical_features = ['시간당발전량(kWh)', '수평면(w/㎡)', '경사면(w/㎡)', '모듈온도(℃)', '기온(°C)', '습도(%)',
                           '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '일조(hr)',
                           '일사(MJ/m2)', '시정(10m)']
categorical_features = ['전운량(10분위)']
cyclic_features = ['Hour', 'Month']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cyc', 'passthrough', [f + '_sin' for f in cyclic_features] + [f + '_cos' for f in cyclic_features]),
        ('cat', 'passthrough', categorical_features)
    ])

In [ ]:
# Perform cyclical encoding for cyclic features
def cyclical_encode(df, feature, max_val):
    df[feature + '_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
    df[feature + '_cos'] = np.cos(2 * np.pi * df[feature] / max_val)
    df = df.drop(feature, axis=1)
    return df

In [ ]:
# Separate input features and target variable
X_dict = dict()
y_dict = dict()
for b, data in buildings.items():
    # Apply cyclical encoding for each cyclic feature
    for feature in cyclic_features:
        max_val = data[feature].max()
        buildings[b] = cyclical_encode(data, feature, max_val)

    X = data[numerical_features + categorical_features + [f + '_sin' for f in cyclic_features] + [f + '_cos' for f in cyclic_features]]
    y = data['시간당발전량(kWh)']

    # Create the ColumnTransformer for preprocessing
    # One-hot encoding for categorical features
    # Keep the numerical and cyclical features as they are
    label_encoder = LabelEncoder()
    X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
    # Apply the preprocessing to the input features
    X_preprocessed = preprocessor.fit_transform(X)

    # Print the preprocessed shape
    print("Preprocessed shape:", X_preprocessed.shape)

    # Convert the preprocessed data into a PyTorch tensor
    X_tensor = torch.tensor(X_preprocessed, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    X_dict[b] = X_tensor
    y_dict[b] = y_tensor

<ipython-input-7-f0c433e4a07a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-7-f0c433e4a07a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-7-f0c433e4a07a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (18672, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (18672, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)


<ipython-input-7-f0c433e4a07a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-7-f0c433e4a07a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-7-f0c433e4a07a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

Shape checking

In [ ]:
X_tensor.shape

torch.Size([19584, 17])

In [ ]:
y_tensor.shape

torch.Size([19584])

Since we are using LSTM for the prediction, we create sequences for training. We use the past 7 days' data to predict the next 24-hour solar power generation.

In [ ]:
# Define the sequence length (past week's data)
sequence_length = 24 * 7  # 24 hours * 7 days

seq_dict = defaultdict(list)
lab_dict = defaultdict(list)
for b in buildings.keys():
    X_tensor = X_dict[b]
    y_tensor = y_dict[b]
    # Create sequences and labels
    sequences = []
    labels = []
    for i in range(len(X_tensor) - sequence_length - 24):  # Subtract 24 to account for predicting the next 24 hours
        seq = X_tensor[i:i+sequence_length]
        label = y_tensor[i+sequence_length:i+sequence_length+24]  # Next 24 hours
        sequences.append(seq)
        labels.append(label)

    seq_dict[b] = torch.stack(sequences)
    lab_dict[b] = torch.stack(labels)

Shape checking

In [ ]:
seq_dict[b].shape, lab_dict[b].shape

(torch.Size([19392, 168, 17]), torch.Size([19392, 24]))

# Build Model

We first define custom PyTorch dataset and build model for this task.

In [ ]:
# Define a custom PyTorch dataset
class SolarDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

Set hyperparameters for the model.

In [ ]:
# Set the device for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
batch_size = 64
num_epochs = 100
input_size = 17  # Number of features
hidden_size = 64
num_layers = 5
output_size = 24  # Single value prediction

Set the criterion as MSELoss() to get RSME error.

In [ ]:
PATH = "/content/drive/MyDrive/ict-2023/output"
modelsdict = dict()
optimizers = dict()
# Define the loss function and optimizer
criterion = nn.MSELoss()

for b in buildings.keys():
    modelsdict[b] = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
    optimizers[b] = optim.Adam(modelsdict[b].parameters(), lr=0.001)

Now, we split the dataset into train and test set. We store each train-test sets pair to a seperate dictionary for convenience.

In [ ]:
train_dict = dict()
val_dict = dict()
test_dict = dict()

for b in buildings.keys():
    # Split the data into train, validate and test sets (8:1:1)
    X_train, X_test, y_train, y_test = train_test_split(seq_dict[b], lab_dict[b], test_size=0.2, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

    # Create instances of the custom dataset for train and test sets
    train_dataset = SolarDataset(X_train, y_train)
    val_dataset = SolarDataset(X_val, y_val)
    test_dataset = SolarDataset(X_test, torch.zeros_like(y_test))  # Use zero-filled labels for the test set

    # Create PyTorch DataLoaders for train and test sets
    train_dict[b] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dict[b] = DataLoader(test_dataset, batch_size=batch_size)
    val_dict[b] = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
for b in buildings.keys():
    print(f"====== Training models for {b}======")

    # dataloaders, model, and optimzer for each builing
    train_dataloader = train_dict[b]
    val_dataloader = val_dict[b]

    model = modelsdict[b]
    optimizer = optimizers[b]


    checkpoint_path = os.path.join(PATH, b+'-checkpoint.pt')
    for epoch in tqdm.tqdm(range(0, num_epochs)):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels)) #RMSE Loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print the average loss for the epoch
        train_loss = running_loss / len(train_dataloader)

        model.eval()
        running_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Compute loss
                loss = torch.sqrt(criterion(outputs, labels))

                running_loss += loss.item()
        val_loss = running_loss / len(val_dataloader)

        if epoch%10==0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Train loss: {train_loss} Validation Loss: {val_loss}")


    # Save the model checkpoint
    torch.save(model.state_dict(), checkpoint_path)
    modelsdict[b] = model


====== Training models for 중앙도서관======


  1%|          | 1/100 [00:01<01:49,  1.11s/it]

Epoch [1/100] Train loss: 1.9476096497641668 Validation Loss: 1.967868697258734


 11%|█         | 11/100 [00:12<01:53,  1.28s/it]

Epoch [11/100] Train loss: 1.8497571724432487 Validation Loss: 1.8752331541430565


...

 81%|████████  | 81/100 [01:34<00:24,  1.27s/it]

Epoch [81/100] Train loss: 7.9742134137408724 Validation Loss: 8.005388690579322


 91%|█████████ | 91/100 [01:45<00:09,  1.09s/it]

Epoch [91/100] Train loss: 7.976165922580923 Validation Loss: 8.00676575014668


100%|██████████| 100/100 [01:55<00:00,  1.16s/it]


Testing the models...

In [ ]:
for b in buildings.keys():
    model = modelsdict[b]
    test_dataloader = test_dict[b]

    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels))

            test_loss += loss.item()

    # Print the test loss
    test_loss /= len(test_dataloader)
    print(f"[{b}] Test Loss (RSME): {test_loss}")


[중앙도서관] Test Loss (RSME): 8.266879927727484
[시설관리동] Test Loss (RSME): 30.730585036739225
[산업협력관] Test Loss (RSME): 14.933247796420392
[동물실험동] Test Loss (RSME): 10.406225173704085
[신재생에너지동] Test Loss (RSME): 17.96331282584898
[기숙사B동] Test Loss (RSME): 12.032811921218347
[대학C동] Test Loss (RSME): 3.807737696555353
[LG도서관] Test Loss (RSME): 11.110817847713347
[중앙창고] Test Loss (RSME): 7.6594174292779735
[학생회관] Test Loss (RSME): 11.639562883684713
[학사과정] Test Loss (RSME): 5.748942590528919
[삼성환경동] Test Loss (RSME): 16.105365291718513
[축구장] Test Loss (RSME): 34.38519237887475
[다산빌딩] Test Loss (RSME): 4.416617885712655


# The Next 24-hour Prediction

In [ ]:
CHECKPOINT_PATH = "/content/drive/MyDrive/ict-2023/output"

In [ ]:
glob.glob(CHECKPOINT_PATH+"/*-checkpoint.pt")[0].split("/")[-1].split(".")[0][:-11]

'중앙도서관'

In [ ]:
# Load the saved model checkpoint
model_dict = dict()
for path in glob.glob(CHECKPOINT_PATH+"/*-checkpoint.pt"):
    building = path.split("/")[-1].split(".")[0][:-11]
    model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
    model.load_state_dict(torch.load(path))
    model_dict[building] = model

In [ ]:
model_dict.keys()

dict_keys(['중앙도서관', '시설관리동', '산업협력관', '동물실험동', '신재생에너지동', '기숙사B동', '대학C동', 'LG도서관', '중앙창고', '학생회관', '학사과정', '삼성환경동', '축구장', '다산빌딩'])

The prediction based on the last 7 days' data

In [ ]:
pred_PATH = "/content/drive/MyDrive/ict-2023/pred"
data_all = dict() # dataframe for display
for b in buildings.keys():
    valid_sequences = []
    valid_sequences.append(X_dict[b][-24*7:]) # predict based on the last 7 days' data
    valid_sequences = torch.stack(valid_sequences)
    valid_sequences.shape

    model = model_dict[b]
    model.eval()
    prediction = model(valid_sequences.to(device))

    # Convert the predictions tensor to a numpy array
    predictions_array = prediction.detach().cpu().numpy()

    #convert negative value to positive
    rounded_data = np.maximum(predictions_array, 0.0).reshape(-1,1)

    data_all[b] = rounded_data.flatten()

    # Create a DataFrame from the predictions array
    predictions_df = pd.DataFrame(rounded_data, columns=['Prediction'])
    predictions_df.to_csv(os.path.join(pred_PATH, b+".csv"), index=False)

In [ ]:
pd.DataFrame(data_all)

,중앙도서관,시설관리동,산업협력관,동물실험동,신재생에너지동,기숙사B동,대학C동,LG도서관,중앙창고,학생회관,학사과정,삼성환경동,축구장,다산빌딩
0,0.410308,0.000000,1.598031,0.314666,0.000000,0.212535,0.000000,0.094003,0.000000,0.625985,0.000000,0.775297,0.127578,4.225000
1,0.000000,0.858850,1.589110,0.194358,1.230512,0.409028,0.101958,0.162911,0.000000,0.000000,0.000000,0.272356,0.342777,4.284652
2,0.050798,0.928114,2.304653,0.239095,1.982867,0.414361,0.000000,0.546031,0.000000,0.131131,0.000000,0.000000,0.572446,4.360099
3,0.386372,0.636801,4.032032,0.236660,0.812991,0.148456,0.000000,0.435896,0.000000,0.000000,0.450174,0.553008,0.000000,4.536553
4,0.172708,0.529407,6.430432,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.712194,2.004209,0.000000,4.396126
5,0.040247,0.468408,9.285086,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006950,0.261864,1.355920,0.000000,4.501468
6,0.634839,0.000000,11.668193,0.022809,1.277623,0.420197,0.000000,0.985478,0.000000,0.516268,0.012003,0.000000,0.000000,4.429533
7,3.007630,0.000000,14.242100,1.581088,5.849726,4.070512,0.598521,2.834999,0.964635,0.967814,0.451629,0.069635,2.634092,4.544599
8,5.813304,0.000000,16.963636,3.647962,11.262098,10.315152,1.951701,6.311241,2.101328,5.836489,2.236648,6.008681,10.012327,4.591637
9,9.215339,8.666004,19.673069,8.344604,16.117104,17.195417,2.825405,13.639823,2.958558,15.999930,5.418149,14.827602,19.331207,4.645003


In [ ]:
all = pd.DataFrame(data_all)
all.to_csv("/content/drive/MyDrive/ict-2023/pred/merged.csv", index=False)